# Evaluacion de mejor funcion de fitness

## Librerias

In [ ]:
from maze import MazeGenerator
from path_generator import PathGenerator
from genetic_algorithm import GeneticAlgorithm
from gen_functions import fitness, fitness_exp, fitness_log, fitness_sqrt, crossover, mutation
import matplotlib.pyplot as plt
import numpy as np

## Constantes

In [ ]:
EPOCHS = 1000
POPULATION = 50
MUTATION_RATE = 0.1
DIM = 51

FITNESS_FUNCTIONS = {
    "fitness": fitness,
    "fitness_sqrt": fitness_sqrt,
    "fitness_exp": fitness_exp,
    "fitness_log": fitness_log
}

NUM_MAZES = 10

## Generar laberintos

In [ ]:
mazes = []

for i in range(NUM_MAZES):
    maze = MazeGenerator(DIM, DIM)
    maze_dict = maze.get_maze_dict()
    mazes.append(maze_dict)

## Comparar fitness

In [ ]:
results = {func_name: [] for func_name in FITNESS_FUNCTIONS.keys()}
for maze_index, maze_dic in enumerate(mazes):
    print(f"--- Maze {maze_index + 1} ---")

    # Preparar el generador de caminos compartido para todos los algoritmos en este laberinto
    maze_matrix = maze_dic["matrix"]
    entry_position = maze_dic["entry"]
    exit_position = maze_dic["exit"]
    path_gen = PathGenerator(maze_matrix, entry_position)
    path_gen.generate_paths(POPULATION)
    initial_paths = path_gen.get_paths()

    # Ejecutar algoritmo genético para cada función de fitness
    for func_name, fitness_func in FITNESS_FUNCTIONS.items():
        print(f"Ejecutando con función de fitness: {func_name}")
        
        # Crear instancia de algoritmo genético
        genetic = GeneticAlgorithm(initial_paths, maze_dic, MUTATION_RATE, EPOCHS, POPULATION, fitness_func, crossover, mutation)
        
        # Ejecutar el algoritmo y recolectar resultados
        generations, fitness_history, solution_found, final_epoch = genetic.run()
        if solution_found:
            print(f"Solución encontrada en la generación {final_epoch}")
            correct_solutions_count = maze_dic["solutions"][exit_position]
        else:
            print("No se encontró solución")
            correct_solutions_count = 0
        print("-"*50)

        # Guardar los resultados en el diccionario
        results[func_name].append({
            "maze_index": maze_index,
            "solution_found": solution_found,
            "generation_solution_found": final_epoch if solution_found else None,
            "correct_solutions_count": correct_solutions_count
        })

## Resultados

In [ ]:
# Comparativa de generación de primera solución entre funciones de fitness
for func_name, func_results in results.items():
    generations_to_solution = [result["generation_solution_found"] for result in func_results if result["solution_found"]]
    plt.plot(range(len(generations_to_solution)), generations_to_solution, marker='o', label=func_name)

plt.title("Generación de Primera Solución por Función de Fitness")
plt.xlabel("Número de Laberinto")
plt.ylabel("Generación de Primera Solución")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Comparativa de cantidad de soluciones correctas al finalizar entre funciones de fitness
for func_name, func_results in results.items():
    correct_solutions = [result["correct_solutions_count"] for result in func_results]
    plt.plot(range(len(correct_solutions)), correct_solutions, marker='o', label=func_name)

plt.title("Cantidad Total de Soluciones Correctas al Finalizar")
plt.xlabel("Número de Laberinto")
plt.ylabel("Cantidad de Soluciones Correctas")
plt.legend()
plt.grid(True)
plt.show()